In [1]:
import pandas as pd
import numpy as np

# Define the number of samples per class
n_samples_per_class = 500

# --- Define Sensor Profiles for Each Emotion ---

# Profile 1: "happy" (Relaxed)
happy_data = pd.DataFrame({
    # High, stable HRV
    'hrv': np.random.normal(loc=75, scale=5, size=n_samples_per_class),
    # Low ambient noise (e.g., quiet focus or pleasant music)
    'noise_db': np.random.normal(loc=40, scale=5, size=n_samples_per_class),
    # Low movement (calm, not fidgeting)
    'movement': np.random.normal(loc=0.15, scale=0.05, size=n_samples_per_class),
    # Comfortable, bright light
    'light_lux': np.random.normal(loc=500, scale=50, size=n_samples_per_class),
    # Comfortable temperature
    'temperature_c': np.random.normal(loc=23, scale=1, size=n_samples_per_class),
    'Emotion': 'happy'
})

# Profile 2: "plain" (Neutral / Focused)
plain_data = pd.DataFrame({
    # Medium, stable HRV
    'hrv': np.random.normal(loc=55, scale=5, size=n_samples_per_class),
    # Very low ambient noise (e.g., silent library)
    'noise_db': np.random.normal(loc=35, scale=3, size=n_samples_per_class),
    # Very low movement (sitting still)
    'movement': np.random.normal(loc=0.05, scale=0.02, size=n_samples_per_class),
    # Standard indoor light
    'light_lux': np.random.normal(loc=400, scale=50, size=n_samples_per_class),
    # Comfortable temperature
    'temperature_c': np.random.normal(loc=24, scale=1, size=n_samples_per_class),
    'Emotion': 'plain'
})

# Profile 3: "sad" (Stressed / Fatigued)
sad_data = pd.DataFrame({
    # Low, erratic HRV (key stress indicator)
    'hrv': np.random.normal(loc=40, scale=7, size=n_samples_per_class),
    # High or distracting noise
    'noise_db': np.random.normal(loc=65, scale=10, size=n_samples_per_class),
    # High movement (fidgeting, restless)
    'movement': np.random.normal(loc=0.8, scale=0.3, size=n_samples_per_class),
    # Often in low light (studying late, fatigued)
    'light_lux': np.random.normal(loc=250, scale=50, size=n_samples_per_class),
    # Uncomfortable temp (e.g., room is too hot)
    'temperature_c': np.random.normal(loc=28, scale=2, size=n_samples_per_class),
    'Emotion': 'sad'
})

# --- Combine and Shuffle the Data ---

# Concatenate all dataframes
df = pd.concat([happy_data, plain_data, sad_data], ignore_index=True)

# Shuffle the dataset randomly
df_shuffled = df.sample(frac=1).reset_index(drop=True)

# Clean up data to avoid nonsensical values (e.g., negative light)
df_shuffled['hrv'] = df_shuffled['hrv'].clip(lower=25)
df_shuffled['noise_db'] = df_shuffled['noise_db'].clip(lower=20)
df_shuffled['movement'] = df_shuffled['movement'].clip(lower=0)
df_shuffled['light_lux'] = df_shuffled['light_lux'].clip(lower=50)
df_shuffled['temperature_c'] = df_shuffled['temperature_c'].clip(lower=18, upper=35)

# Save to a CSV file
df_shuffled.to_csv('simulated_emogotchi_dataset.csv', index=False)

print("Simulated dataset created and saved as 'simulated_emogotchi_dataset.csv'")
print(df_shuffled.head())

Simulated dataset created and saved as 'simulated_emogotchi_dataset.csv'
         hrv   noise_db  movement   light_lux  temperature_c Emotion
0  74.106022  37.235226  0.146061  432.905869      24.562857   happy
1  68.447969  34.404698  0.152434  606.284178      21.273186   happy
2  70.634204  36.013230  0.153419  504.260959      23.011788   happy
3  80.535296  41.311954  0.111696  542.669646      23.813441   happy
4  37.535198  72.609923  0.788155  249.717724      29.370973     sad


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Load the dataset you just created
df = pd.read_csv('simulated_emogotchi_dataset.csv')

# 1. Define Features (X) and Target (y)
features = ['hrv', 'noise_db', 'movement', 'light_lux', 'temperature_c']
target = 'Emotion'

X = df[features]
y = df[target]

# 2. Split into Training and Testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 3. Initialize and Train the Model
print("Training the Random Forest model...")
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# 4. Evaluate the Model
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Model trained! Accuracy on simulated test data: {accuracy * 100:.2f}%")

Training the Random Forest model...
Model trained! Accuracy on simulated test data: 99.67%


In [6]:
def get_system_action(emotion_prediction, current_sensor_data):
    """
    Decides on an action based on the ML prediction and sensor readings.
    
    :param emotion_prediction: (str) The output from your model, e.g., "sad"
    :param current_sensor_data: (dict) A dictionary of the *current* sensor values
    :return: (list) A list of action commands
    """
    actions_to_take = []
    
    # --- Rules for "sad" (Stressed) State ---
    if emotion_prediction == 'sad':
        print("Model prediction: 'sad'. Checking environment...")
        
        # Light rule
        if current_sensor_data['light_lux'] < 300:
            actions_to_take.append("turn up light")
            
        # Temperature rule (e.g., it's too hot)
        if current_sensor_data['temperature_c'] > 27:
            actions_to_take.append("switch on AC")
            
        # Temperature rule (e.g., it's too cold)
        if current_sensor_data['temperature_c'] < 21:
            actions_to_take.append("switch off AC")

    # --- Rules for "happy" (Relaxed) State ---
    elif emotion_prediction == 'happy':
        print("Model prediction: 'happy'. Checking environment...")
        
        # Light rule (e.g., light is too bright/harsh)
        if current_sensor_data['light_lux'] > 800:
            actions_to_take.append("turn down light")
            
    # --- Rules for "plain" (Neutral) State ---
    else:
        print("Model prediction: 'plain'. No action needed.")
        actions_to_take.append("no_action")
        
    return actions_to_take

# --- --- ---
# --- SIMULATION EXAMPLE ---
# --- --- ---

# 1. Get a new, unseen data point (simulating your ESP32)
new_data_point = {
    'hrv': 38,
    'noise_db': 70,
    'movement': 1.1,
    'light_lux': 220,  # <-- Notice light is low
    'temperature_c': 29 # <-- Notice temp is high
}

# Convert to the format the model expects (a 2D array)
data_for_model = [list(new_data_point.values())]

# 2. Use the ML model to predict the emotion
predicted_emotion = model.predict(data_for_model)[0] # Output will be 'sad'

# 3. Feed the prediction AND sensor data into the rules engine
final_actions = get_system_action(predicted_emotion, new_data_point)

# 4. Get the final command
print(f"\nFinal Actions: {final_actions}")

Model prediction: 'sad'. Checking environment...

Final Actions: ['turn up light', 'switch on AC']


c:\Users\dongh\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
